In [8]:
import flopy as fp
import numpy as np
import geopandas as gp
import pandas as pd
import matplotlib.pyplot as plt
from flopy.utils.gridgen import Gridgen 
import shapely
from flopy.utils.gridintersect import GridIntersect

In [4]:
model_dir = "workingDis"
model_name="dis_test"
exe_name="../../exe.mf6"

In [49]:
def Imreduction (imar, nrow, ncol):
    
    """
    This function reduce the dimensionality of an image.
    An array of the image must be provided by the user, same for the number of row and col of the new image
    No data should be specified as "None" type
    return a new array
    """
    
    #first define the ratio between the given and the desired resolution in x and y
    facR=imar.shape[0]/nrow
    facC=imar.shape[1]/ncol
    
    # go through the entire grid (nrow,ncol) and use the mean of all the values in the array image
    # in a specific cell of the grid (i,o)
    new_imar=[]
    for i in range(nrow):
        for o in range(ncol):
            new_val=np.nanmean(imar[np.round(i*facR).astype(int):np.round(i*facR+facR).astype(int),
                                    np.round(o*facC).astype(int):np.round(o*facC+facC).astype(int)])
            new_imar.append(new_val)
    
    # create an array of the new image array
    new_imar = np.array(new_imar)
    return np.array(pd.DataFrame(new_imar.reshape(nrow,ncol)).fillna(0))

In [79]:
def gp2idomain (gp,grid,idomain,area=3):
    
    '''
    This function attribute active values to cells given a certain geopandas object and a grid (flopy.discretization) with idomain
    the tolerance is a value that determine at which level a cell will be counted as intersected by the polygon 
    (3 for example mean that only cells that have 1/3 of their area intersected by the polygon will be accounted)
    '''
    
    ix = GridIntersect(grid)
    
    if area == 0:
        result = ix.intersect_polygon(gp.geometry[0])
    
    if area >= 1:
        result = ix.intersect_polygon(gp.geometry[0])
        result = result[result.areas>(np.max(result.areas)/area)]
    
        
    
    for irow, icol in result2.cellids:
        idomain[irow*nrow+icol]=1
        
    return idomain

In [6]:
#spatial parameters

x0 = 660000 # left min
x1 = 705000 # right max
Lx = x1-x0
y0 = 6150000 # bottom min
y1 = 6205000 # top max
Ly = y1-y0
nlay = 1
ncol = 50
nrow = 50
delr = np.ones(ncol)*(Lx/ncol)
delc = np.ones(nrow)*(Ly/nrow)
botm = -50
idomain = np.zeros((nrow*ncol))

R_path="../data/limiteModeleRoussillon_poly.shp" # path to the shp of the aquifer

In [80]:
# import a mnt and extract topography data

from PIL import Image
im = Image.open("../../../../../TRAVAIL DE MASTER/data/MNT_demo_5m.tif") # add path to MNT.tif
imar = np.array(im)
imar[imar==0]=None

top = Imreduction(imar,nrow,ncol)

In [81]:
#cells inside the aquifer become active

R = gp.read_file(R_path) # import shapefile with geopandas
grid = fp.discretization.StructuredGrid(delc,delr,xoff=x0,yoff=y0)

idomain = gp2idomain(R,grid,idomain,area=3)

In [85]:
# basic modules
sim = fp.mf6.MFSimulation(sim_name='theis_mf6', version='mf6', exe_name=exe_name, 
                             sim_ws=model_dir)
gwf = fp.mf6.ModflowGwf(sim, modelname=model_name,
                           model_nam_file='{}.nam'.format(model_name))
dis = fp.mf6.ModflowGwfdis(gwf, nlay=nlay, nrow=nrow, ncol=ncol,
                              delr=delr, delc=delc,
                              top=top, botm=botm,xorigin=x0,yorigin=y0,idomain=idomain)
tdis = fp.mf6.ModflowTdis(sim, time_units='DAYS',perioddata=[[1.0, 1, 1.]])
ims  = fp.mf6.ModflowIms(sim, print_option='SUMMARY', complexity='simple')

In [88]:
# initial conditions
ic   = fp.mf6.ModflowGwfic(gwf)

In [93]:
# nodeflowproperty
kh = 1e-4
npf  = fp.mf6.ModflowGwfnpf(gwf, k=kh, save_specific_discharge=True,save_flows=True)